In [1]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
with open("/home/trnmah/UET-project/summarization/drive-download-20240727T015440Z-001/train_data_new.jsonl", 'r') as f:
	data = f.readlines()
	data = [json.loads(d) for d in data]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base-vietnews-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base-vietnews-summarization")

In [3]:
data[0]['summary']

'Các giáo viên bộ môn giàu kinh nghiệm đã có những chia sẻ giúp học sinh đạt kết quả cao trong kỳ thi sắp tới. Cần phải giữ sức khỏe ổn định bằng cách sinh hoạt điều độ; về việc ôn thi, không nên tập trung vào học kiến thức mới nữa, mà nên ưu tiên luyện tập chiến thuật làm bài thi. Đối với môn Ngữ văn, cần nắm chắc cấu trúc đề, dạng câu hỏi theo chuyên đề, rà soát lại các kiến thức cơ bản trong chương trình Ngữ văn lớp 12. Chiến thuật làm môn Toán là: Dễ trước, khó sau, nháp cẩn thận, khoanh vùng rõ ràng, tô đáp án cẩn thận. Đối với môn tiếng Anh,  cần tối ưu hóa điểm số dựa trên mục tiêu, làm sau và dành nhiều thời gian cho hai dạng bài khó là đọc hiểu và đọc điền. Khi làm bài Khoa học xã hội, câu dễ cần làm nhanh và chắc, câu khó làm sau, tránh căng thẳng dẫn đến những sai sót không đáng có. '

In [24]:
sum(tokenizer(data[0]['single_documents'][2]['raw_text']).attention_mask)

1715

In [8]:
data[1]['single_documents']

[{'title': "Biến đổi khí hậu khiến châu Âu 'điêu đứng' vì nắng nóng ",
  'anchor_text': 'Nhiệt độ kỷ lục mới được ghi nhận tại Vương quốc Anh, quốc gia nổi tiếng với khí hậu ôn hòa và nhiều mưa. Đây là một phần trong đợt sóng nhiệt nóng và khô bất thường đang càn quét khắp châu Âu, ảnh hưởng trực tiếp tới sức khỏe con người, cơ sở hạ tầng và môi trường - làm dấy lên nhiều lo ngại về biến đổi khí hậu. ',
  'raw_text': 'Anh lần đầu ghi nhận nắng nóng trên 40 độ C\nNhiệt kế đã chỉ 40,3 độ C tại hạt Lincolnshire, miền trung nước Anh vào lúc 4 giờ chiều (giờ địa phương), cao hơn gần 2 độ so với nhiệt độ kỷ lục cũ năm 2019. Tới chiều ngày 19/7, đã có 34 địa phương phá vỡ mức nhiệt kỷ lục trước đó.\nNền nhiệt cao tác động xấu đến nhiều mặt của đời sống người dân đất nước này, từ giao thông tới y tế và giáo dục. Nhiều gia đình, doanh nghiệp nhỏ và cả các tòa nhà công cộng như bệnh viện tại Anh không có hệ thống điều hòa không khí. Sở Cứu hỏa London đã ra thông báo khẩn khi nhiều vụ cháy xảy ra

In [17]:
data[0]['single_documents'][0]

{'title': 'Lời dặn dò tâm huyết của thầy cô trước khi học sinh bước vào kỳ thi THPT ',
 'anchor_text': 'Trong 2 ngày 7 và 8/7/2022, hơn 1 triệu sĩ tử trên cả nước sẽ bắt đầu kỳ thi THPT Quốc gia năm 2022. Bước vào kỳ thi quan trọng này, tâm lý các thí sinh không tránh khỏi lo lắng. Giáo viên Hệ thống giáo dục Hocmai đưa ra những lời dặn dò hết sức tâm huyết và ý nghĩa dành cho các học trò. ',
 'raw_text': 'TS Trịnh Thu Tuyết: Rà soát lại một lần nữa những đơn vị kiến thức cơ bản\nChỉ còn vài ngày nữa, các em thực sự bước vào kì thi Tốt nghiệp THPT, với môn thi đầu tiên là môn Ngữ văn, áp lực với kì thi nói chung, với môn Ngữ văn nói riêng là không tránh khỏi. Tuy nhiên, đây cũng là thời điểm "lúa sắp gặt, trái sắp chín", công sức bao lâu sắp hiện trong thành quả của một kì thi thành công, và cô chỉ muốn nhắc các em vài điều cụ thể sau đây:\n1. Thay vì lo lắng và áp lực, các em cần tích cực rà soát lại một lần nữa những đơn vị kiến thức cơ bản trong chương trình Ngữ văn lớp 12, ghi nhớ 

In [30]:
test1= tokenizer(data[1]['single_documents'][0]['raw_text']+ data[1]['single_documents'][1]['raw_text']+ data[1]['single_documents'][2]['raw_text'], return_tensors='pt')

In [31]:
test1['input_ids'].shape

torch.Size([1, 2992])

In [34]:
tokenizer(data[16]['summary'], return_tensors='pt').input_ids.shape

torch.Size([1, 177])

In [4]:
# create dataset class
from torch.utils.data import Dataset, DataLoader
import torch
class Dataset4Summarization(Dataset):
	def __init__(self, data, tokenizer, max_length=5096, chunk_length =1024, overlap=16):
		self.data = data
		self.tokenizer = tokenizer
		self.max_length = max_length
		self.chunk_length = chunk_length
		self.overlap = overlap

	def __len__(self):
		return len(self.data)

	def join_single_documents(self, single_documents):
		text = ""
		for documents in single_documents:
			text += documents['raw_text'] + " "
		return text
	
	def chunking(self, text):
		chunks = []
		for i in range(0, self.max_length, self.chunk_length-self.overlap):
			chunks.append(text[i:i+self.chunk_length])
		return chunks

	def __getitem__(self, idx):
		sample = self.data[idx]
		text = self.join_single_documents(sample['single_documents'])
		inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=5096)
		target = self.tokenizer(sample['summary'], return_tensors='pt', padding='max_length', truncation=True, max_length=256)
		list_chunk = self.chunking(inputs['input_ids'].squeeze())
		list_attention_mask = self.chunking(inputs['attention_mask'].squeeze())


		del text
		del sample
		return {
			'list_input_ids': list_chunk,
			'list_att_mask' : list_attention_mask,
			'target': target['input_ids'].squeeze()
		}

In [5]:
dataset = Dataset4Summarization(data, tokenizer)

In [6]:
train_loader = DataLoader(dataset, batch_size=2)

In [48]:
for iter in train_loader:
	print(iter['list_input_ids'][0].shape)
	print(len(iter['list_input_ids']))
	break

torch.Size([2, 1024])
6


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
test1_input = tokenizer(data[1]['single_documents'][0]['raw_text'], return_tensors='pt', padding='max_length', truncation=True, max_length=1024)

In [4]:
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [16]:
ja = model.generate(input_ids=test1_input['input_ids'].to('cuda'), max_length=256, num_beams=4, early_stopping=True)

In [19]:
ja[0]

tensor([   0, 3556, 2369,  128,  714, 1956,  411,  845,  823, 1119,    5,  718,
         170,  204, 2534, 4214, 1956, 1620,  674,  195,  690,  996, 1220,    6,
           1], device='cuda:0')

In [6]:
model(input_ids=test1_input['input_ids'].to('cuda'), attention_mask=test1_input['attention_mask'].to('cuda'), labels=test1_input['input_ids'].to('cuda'))

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 

In [61]:
tokenizer.decode(ja[0], skip_special_tokens=True)

'Lần đầu tiên trong 6 năm, Anh ghi nhận nắng nóng trên 40 độ C, mức cao kỷ lục chưa từng có năm 2019.'

In [60]:
tokenizer.special_tokens_map

{'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>',
 'additional_special_tokens': ['<extra_id_0>',
  '<extra_id_1>',
  '<extra_id_2>',
  '<extra_id_3>',
  '<extra_id_4>',
  '<extra_id_5>',
  '<extra_id_6>',
  '<extra_id_7>',
  '<extra_id_8>',
  '<extra_id_9>',
  '<extra_id_10>',
  '<extra_id_11>',
  '<extra_id_12>',
  '<extra_id_13>',
  '<extra_id_14>',
  '<extra_id_15>',
  '<extra_id_16>',
  '<extra_id_17>',
  '<extra_id_18>',
  '<extra_id_19>',
  '<extra_id_20>',
  '<extra_id_21>',
  '<extra_id_22>',
  '<extra_id_23>',
  '<extra_id_24>',
  '<extra_id_25>',
  '<extra_id_26>',
  '<extra_id_27>',
  '<extra_id_28>',
  '<extra_id_29>',
  '<extra_id_30>',
  '<extra_id_31>',
  '<extra_id_32>',
  '<extra_id_33>',
  '<extra_id_34>',
  '<extra_id_35>',
  '<extra_id_36>',
  '<extra_id_37>',
  '<extra_id_38>',
  '<extra_id_39>',
  '<extra_id_40>',
  '<extra_id_41>',
  '<extra_id_42>',
  '<extra_id_43>',
  '<extra_id_44>',
  '<extra_id_45>',
  '<extra_id_46>',
  '<extra_id_47>',
 

In [63]:
xx = torch.randn(2, 3, 4)
xy = torch.randn(2, 3, 4)
a = [xx, xy]
torch.cat(a, dim=1).shape


torch.Size([2, 6, 4])

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, fused=True )
critertion = torch.nn.CrossEntropyLoss().to(device)

In [10]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [13]:
def train_fn(model, model1, train_loader, optimizer, scheduler, criterion, epochs, device):
	model.train()
	for epoch in range(epochs):
		for iter in train_loader:
			summaries = []
			model.zero_grad(set_to_none=True)
			inputs = iter['list_input_ids']
			att_mask = iter['list_att_mask']
			target = iter['target'].to(device)
			for i in range(len(inputs)):
				inputs[i] = inputs[i].to(device)
				# att_mask[i] = att_mask[i].to(device)
				summary = model.generate(inputs[i], max_length=64, num_beams=4, early_stopping=True)
				summaries.append(summary)
			summaries = torch.cat(summaries, dim=1).to(device)
			# summaries = model.generate(summaries, max_length=256, num_beams=4)
			# del inputs
			if summaries.shape[1] > 256:
				summaries = summaries[:, :256]
				att_mask = torch.ones((summaries.shape[0], 256)).to(device)
			else:
				att_mask = torch.ones((summaries.shape[0], summaries.shape[1])).to(device)
				summaries = torch.nn.functional.pad(summaries, (0, 256-summaries.shape[1]), value=0)
				att_mask = torch.nn.functional.pad(att_mask, (0, 256-att_mask.shape[1]), value=0)
			
			summaries = model1(summaries,attention_mask=att_mask, labels=target)
			loss = summaries.loss

			torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
			loss.backward()
			optimizer.step()
			
			print(loss.item())
		scheduler.step()
		print(f'Epoch {epoch} done')

In [14]:
train_fn(model, train_loader, optimizer, scheduler, critertion, 1, device)

torch.Size([2, 256])
torch.Size([2, 56])


RuntimeError: "host_softmax" not implemented for 'Long'

In [41]:
xx = torch.randn(2,39)
padded = torch.nn.functional.pad(xx, (0,  56-39), value=0)
padded.shape

torch.Size([2, 56])

In [26]:
tokenizer(tokenizer.special_tokens_map['pad_token'])

{'input_ids': [0, 1], 'attention_mask': [1, 1]}

In [28]:
tokenizer.decode(1)

'</s>'